This code replicates methods from @kaandorp_optimal_2007.


In [ ]:
import logging
import copy
import datetime
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy.stats import lognorm
from scipy import signal
import plotly.graph_objs as go
import plotly.offline as pyo
import unittest
import functions as fn
from itertools import chain, combinations
import plotly.graph_objs as go
import copy

logging.basicConfig(filename='logs.txt', encoding='utf-8', level=logging.DEBUG)

# """
# Function to calculate the convolution of two arrays.
# 
# Args:
#     a (numpy.ndarray): The first array to be convolved.
#     b (numpy.ndarray): The second array to be convolved.
# 
# Returns:
#     numpy.ndarray: The convolution of the two input arrays.
# """
def convolve(a, b):
    
    # Initialize an empty array to store the result.
    c = np.array([])
    
    # Compute the convolution of the two arrays.
    for i in range(len(a)):
        # Get subsets of array expanded to the right.
        a_sub = a[0:i + 1].copy()
        b_sub = b[0:i + 1].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
    
    for i in range(1,len(a)):
        # Get subsets of array collapse from the right.
        a_sub = a[i:].copy()
        b_sub = b[i:].copy()
        # Reverse b.
        b_rev = b_sub[::-1]
        # Compute the dot product of a and b_rev.
        c = np.append(c, np.dot(a_sub, b_rev))
        
    return c


# """
# Function to convolve a distribution with itself n times.
# 
# Args:
#     a (numpy.ndarray): The distribution to be convolved.
#     n (int): The number of times to convolve the distribution with itself.
# 
# Returns:
#     numpy.ndarray: The convolution of the input distribution with itself n times.
# """
def convolve_n(a, n):
        
    # Initialize an empty array to store the result.
    c = np.array([])
    
    # If n is 0, return an array of zeros with length equal to the length of a, except for the first element which is 1.
    if n == 0:
        c = np.array(np.zeros(len(a)), dtype=np.float64)
        c[0] = 1
        return c
    
    # Convolve the distribution with itself n times.
    for i in range(n):
        # If this is the first iteration, set c equal to a.
        if i == 0:
            c = a
        # Otherwise, convolve c with a.
        else:
            c = np.convolve(c, a)
            
    return c
# 
# """
# Function to create an array of zero arrays according to a given shape array.
# 
# Args:
#      num_zeros (numpy.ndarray): The shape array.
#      l (int): The length of the zeros array.
#  
# Returns:
#      numpy.ndarray: The convolution of the input distribution with itself n times.
# """

def zero_arrays(num_zeros, l):
    result = []
    for n in num_zeros:
        zeros = np.zeros(l)
        result.append([zeros] * n)
    return result

print(f'Zero arrays are: {zero_arrays(np.array([1, 0, 3]), 4)}')

def calc_distr_limit(l):
    return int(max(l+4*l**0.5, 100))
  
class TestConvolve(unittest.TestCase):
    
    def test_convolve(self):
        a = np.array([
            0.4456796414,
            0.160623141,
            0.137676978,
            0.1032577335])

        b = np.array([
            0.006737946999,
            0.033689735,
            0.08422433749,
            0.1403738958])

        expected_output = np.convolve(a, b)
        
        self.assertTrue(np.allclose(convolve(a, b), expected_output))
    
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return [list(item) for item in chain.from_iterable(combinations(s, r) for r in range(len(s)+1))]

def get_v_star(t):
    # Create an initial vector 'u' of zeros with length 't'
    u = np.zeros(t, dtype=np.int64)
    # Set the first element of vector 'u' to -1
    u[0] = -1
    # Set the last element of vector 'u' to 1
    u[-1] = 1
    # Initialize the list 'v_star' with the initial vector 'u'
    v_star = [u]
    # Loop over the length of 'u' minus one times
    for i in range(len(u) - 1):
        # Append the last element of 'u' to the front of 'u'
        u = np.append(u[-1], u)
        # Remove the last element of 'u' to maintain the same length
        u = np.delete(u, -1)
        # Append the updated vector 'u' to the list 'v_star'
        v_star.append(u)
    # Convert the list of vectors 'v_star' into a NumPy array and return it
    return(np.array(v_star))

# Example of function call:
# This will create a 4x4 matrix where each row is a cyclically shifted version of the first row

def generate_search_neighborhood(schedule):
  N = sum(schedule)
  T = len(schedule)
  logging.info(f'The schedule = {schedule}')
  
  # Generate a matrix 'v_star' using the 'get_v_star' function
  v_star = get_v_star(T)
  
  # Generate all possible non-empty subsets (powerset) of the set {0, 1, 2, ..., T-1}
  # 'ids' will be a list of tuples, where each tuple is a subset of indices
  ids = list(powerset(range(T)))

  logging.info(f'Neighborhood size = {len(ids)}')

  # Select the vectors from 'v_star' that correspond to the indices in each subset
  # 'sub_sets' will be a list of lists, where each inner list contains vectors from 'v_star'
  sub_sets = [v_star[i] for i in ids]

  # Sum the vectors within each subset and flatten the result to get a 1-D array
  # 'summed_sets' will be a list of 1-D numpy arrays, where each array is the sum of vectors
  summed_sets = [np.sum(sub_sets[i], axis=0).flatten() for i in range(len(sub_sets))]
  
  neighborhood = np.array([schedule + summed_sets[i] for i in range(len(summed_sets))])
  
  # Create a mask for rows with negative values
  mask = ~np.any(neighborhood < 0, axis=1)

  # Filter out rows with negative values using the mask
  filtered_neighborhood = neighborhood[mask]
  logging.info(f'And the neighborhood is {filtered_neighborhood}')
  return filtered_neighborhood

def generate_small_search_neighborhood(schedule):
  N = sum(schedule)
  T = len(schedule)
  logging.info(f'The schedule = {schedule}')
  
  # Generate a matrix 'v_star' using the 'get_v_star' function
  v_star = get_v_star(T)
  
  neighborhood = np.array([schedule + v_star[i] for i in range(len(v_star))])
  
  # Create a mask for rows with negative values
  mask = ~np.any(neighborhood < 0, axis=1)

  # Filter out rows with negative values using the mask
  filtered_neighborhood = neighborhood[mask]
  logging.info(f'And the neighborhood is {filtered_neighborhood}')
  return filtered_neighborhood

def distribute_patients(n_patients, n_timeslots):
    # Create a list with all slots initially empty
    distribution = [0] * n_timeslots

    # Place patients in timeslots
    for i in range(n_patients):
        # Calculate the slot for each patient
        slot = round(i * n_timeslots / n_patients)
        distribution[slot] = 1

    return distribution

def plot_timeline(slots, title):
    # Create a figure
    fig = go.Figure()

    # Iterate over each timeslot and add a bar for occupied slots
    for i, slot in enumerate(slots):
        fig.add_trace(go.Bar(x=[i], y=[slot], width=0.8, marker_color='black'))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="Timeslots",
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        showlegend=False
    )

    # Set y-axis range
    fig.update_yaxes(range=[0, np.max(slots)])

    # Show the figure
    fig.show()
    
def service_time_with_no_shows(s, q):
  # """
  # Function to adjust a distribution of service times for no-shows
  # 
  # Args:
  #     s (numpy.ndarray): An array with service times.
  #     q (double): The fraction of no-shows.
  # 
  # Returns:
  #     numpy.ndarray: The adjusted array of service times.
  # """
  
  s_adj = s * (1-q)
  s_adj[0] = s_adj[0] + q
  return(s_adj)

def add_lists(short_list, long_list):
  # """
  # This function takes in two lists and returns a new list where each element 
  # is the sum of the elements from the input lists at the corresponding position.
  # If the lists are of different lengths, the shorter list is extended with zeros 
  # to match the length of the longer list.
  # 
  # Parameters:
  # - short_list (list): The shorter list of numbers.
  # - long_list (list): The longer list of numbers.
  # 
  # Returns:
  # - list: A list containing the element-wise sum of the two input lists.
  # """
  
  # Extend the short lists to the length of the long list with zeros
  short_list.extend([0] * (len(long_list) - len(short_list)))
  
  # Sum the elements of the two lists element-wise
  result = [a + b for a, b in zip(short_list, long_list)]
  
  return result

def calculate_rolling_convolution(p_y, s):
  conv_list = s
  limit = len(p_y)
  v = [x * p_y[0] for x in conv_list]
  for i in range(1, limit):
    conv_list = np.convolve(conv_list, s)
    v = add_lists(v, [x * p_y[i] for x in conv_list])
    
  return(v)

def generate_schedules(N):
    """
    Generate all possible ways to distribute N patients into 3 timeslots.
    The function returns a list of lists, where each sublist represents
    a distribution of patients across the 3 timeslots such that the sum of
    the numbers in the sublist is N.

    Parameters:
    N (int): The total number of patients to be distributed.

    Returns:
    List[List[int]]: A list of lists where each sublist contains 3 integers
    representing a possible distribution of patients across the 3 timeslots.
    """
    # This list will hold the result
    schedules = []

    # Iterate over all possible values of 'a' from 0 to N (inclusive)
    for a in range(N + 1):
        # For each value of 'a', iterate over all possible values of 'b' from 0 to N-a (inclusive)
        for b in range(N - a + 1):
            # Compute 'c' such that the sum of 'a', 'b', and 'c' is N
            c = N - a - b
            # Append the combination [a, b, c] to the schedules list
            schedules.append([a, b, c])

    # Return the list of all possible schedules
    return schedules

A schedule with $T$ intervals can have $T$ states. A state of a schedule at interval $t$ is defined by:

\- $p^-_t(i)$, the distribution of the amount of work ($i$) left at the end of the state at interval $t-1$,

\- $w_{tk}(i)$, the distribution of waiting time ($i$) for a patient $k$ in interval $t$

\- $p^+_t(i)$, the probability of the total amount of work ($i$) in interval $t$, ergo: the work left from the previous state plus all work related to arriving patients.

$p^+_t(i)$ is equal to the convolution of the distribution of waiting times of the last patient with the distribution of his service time. The iteration is as follows with K patients scheduled at interval t:

> Step 1: $w_{t0}(i) = p^-_t(i)$ \# The first patient has to wait for all the work leftover from the previous interval

> Step 2: $w_{t1}(i) = w_{t0}(i)*s(i)$ \# The next patient has to wait for the previous patient waiting time and service time

> . . .

> Step K: $p^+_t(i) = w_{t(K-1)}(i)*s(i)$

With $s(i)$ being the distribution of individual service times. **In the examples below we model service times as a Poisson process, because we are calculating in discrete time units.**


In [ ]:
"""
A schedule class with a constructor and a method to calculate the system states.
"""
class Schedule:
    def __init__(self, x, d, s, q, omega):
        """
        Initialize the Schedule class with patient schedules and a service time distribution.

        Args:
            x (list): A list of integers representing the number of patients scheduled to arrive at each time step.
            d (int): An integer representing the length of a time interval.
            s (list): A list of floats representing the probability distribution of service times.
            q (float): A float representing the probability of patients not showing up.
            omega (float): A float representing weight for the waiting cost in the loss function The weight for tardiness equal 1 - omega.
        """
        if not all(isinstance(i, np.integer) and i >= 0 for i in x):
            raise ValueError("All elements in x must be non-negative integers.")
        if not isinstance(d, int) or d <= 0:
            raise ValueError("d must be a positive integer.")
        if not all(isinstance(i, float) and 0 <= i <= 1 for i in s):
            raise ValueError("All elements in s must be floats between 0 and 1.")
        if not isinstance(q, float) and 0 <= q <= 1:
            raise ValueError("q must be a float between 0 and 1.")
        if not isinstance(omega, float) and 0 <= omega <= 1:
            raise ValueError("omegea must be a float between 0 and 1.")

        self.parameters = {'x': x, 'd': d, 's': s, 'q': q, 'omega': omega}
        """ Adjust service times for no-shows. """
        self.parameters['s'] = service_time_with_no_shows(self.parameters['s'], self.parameters['q'])
        self._initialize_system()
            
    def _initialize_system(self):
        """ Initialize the system's internal state. """
        self.state = 0
        length_x = len(self.parameters['x'])
        length_s = len(self.parameters['s'])
        self.system = {
            'p_min': np.zeros((length_x + 1, length_s), dtype=np.float64),
            'p_plus': np.zeros((length_x, length_s), dtype=np.float64),
            'w': [np.zeros((i, length_s), dtype=np.float64) for i in self.parameters['x']],
            'ew': np.zeros(length_x, dtype=np.float64),
            'loss': None
        }
        self.system['p_min'][0][0] = 1
        if(self.parameters['x'][0] > 0): # Only calculate waiting times if there are patients scheduled in the state
                     self.system['w'][0][0] = self.system['p_min'][0].copy()
                     for i in range(1, self.parameters['x'][0]):
                            self.system['w'][0][i] = np.convolve(self.system['w'][0][i-1], self.parameters['s'])[:(len(self.parameters['s']))]
        self._update_p_plus(0)
        self.state = 1
            
    def _update_p_plus(self, state):
        """ Update the p_plus array based on the current state. """
        if self.parameters['x'][state] == 0:
            self.system['p_plus'][state] = self.system['p_min'][state].copy()
        else:
            self.system['p_plus'][state] = np.convolve(self.system['w'][state][-1], self.parameters['s'], mode='full')[:len(self.parameters['s'])]
        logging.info(f"p_plus = {self.system['p_plus'][state]}")
            
    def _calculate_state(self):
        logging.info(f'{datetime.datetime.now()} - State = {self.state}')
        
        """The probability that the amount of work left in the system equals zero just before state t starts is the probablity that the total amount work in state t-1 was less than or equal to the interval length d."""
        logging.info(f'{datetime.datetime.now()} - Calculating p_min in state {self.state}')
        self.system['p_min'][self.state][0] = np.sum(self.system['p_plus'][self.state-1][:(self.parameters['d'] + 1)])
        
        """The probability that the amount of work left in the system equals i just before state t starts is the probablity that the total amount work in state t-1 exceeded the interval length d with amount i."""
        
        self.system['p_min'][self.state][1:(-1*self.parameters['d'])] = self.system['p_plus'][self.state-1][(self.parameters['d'] + 1):]
        
        """The distribution of waiting times of the first patient in state t equals p_min. 
        The distribution of waiting times of the second patient in state t equals the convolution of the distribution  of waiting times of the first patient in state t and the service time distribution. The resulting vector is truncated to the length of the service time distribution."""
        if(self.parameters['x'][self.state] > 0): # Only calculate waiting times if there are patients scheduled in the state
                logging.info(f'{datetime.datetime.now()} - Calculating w[{self.state}][0] in state {self.state}')
                self.system['w'][self.state][0] = self.system['p_min'][self.state].copy()
                logging.info(self.system['w'][self.state][0])
                logging.info(f'{datetime.datetime.now()} - Done')
                for i in range(1, self.parameters['x'][self.state]):
                    logging.info(f'{datetime.datetime.now()} - Calculating w[{self.state}][{i}] in state {self.state}')
                    self.system['w'][self.state][i] = np.convolve(self.system['w'][self.state][i-1], self.parameters['s'])[:(len(self.parameters['s']))]
                    logging.info(self.system['w'][self.state][i])

        """The probablitity that the amount of work left in the system equals i just before state t ends equals the convolution of the waiting time distribution of the last arriving patient and the service time distribution. Unless there are no patients in the state t. In that case the distribution of total work just before t ends is equal to the distribution of work at the beginning of t. The resulting vector is truncated to the length of the service time distribution."""
        logging.info(f'{datetime.datetime.now()} - Calculating p_plus in state {self.state}')
        self._update_p_plus(self.state)
    
    def calculate_system_states(self, until=1):
        """
        Calculate the probabilities of the system being in each state at each time step.

        Args:
            until (int, optional): The state until which to calculate the probabilities. Defaults to 1.
        """
        while self.state < until:
            self._calculate_state()
            self.state += 1
            
    def calculate_loss(self):
        omega = self.parameters['omega']
        # Calculate loss
        tot_wt = self.system['ew'].sum()
        self.calculate_tardiness()
        indices = np.arange(self.system['p_min'][-1].size) 
        exp_tard = (indices * self.system['p_min'][-1]).sum()
        self.system['loss'] = omega * tot_wt / self.parameters['x'].sum() + (1 - omega) * exp_tard
      
    def local_search(self, omega=0.5):
        """
        Perform a local search to optimize a schedule by minimizing the total waiting time and expected tardiness.
        
        The function starts with an initial schedule and iteratively explores neighboring schedules. For each neighboring schedule, it recalculates the system's states and computes the total waiting time and expected tardiness. If a neighboring schedule with a lower objective value (weighted sum of normalized waiting time and expected tardiness) is found, the function updates the current best schedule and continues the search from this new schedule. The search stops when no better neighboring schedule is found.
        
        Parameters:
        omega (float): A weighting factor between 0 and 1 used to balance the trade-off between total waiting time and expected tardiness.
        
        The function follows these steps:
        1. Calculate the initial loss using the total waiting time and expected tardiness.
        2. Store the current schedule and system state as the initial best solution.
        3. Enter a loop to explore neighboring schedules:
           a. Generate a neighborhood of candidate schedules.
           b. For each candidate schedule in the neighborhood:
              i. Update the schedule and reinitialize the system.
              ii. Recalculate the system states and waiting times.
              iii. Calculate the objective value (weighted sum of normalized waiting time and expected tardiness).
              iv. If a better schedule is found, update the best solution and break to generate a new neighborhood.
        4. If no better schedule is found in the neighborhood, end the search.
        5. Set the system to the best found schedule and system state.
        """
        omega = self.parameters['omega']
        # Calculate initial loss
        test_wt = self.system["ew"].sum()
        self.calculate_tardiness()
        indices = np.arange(self.system['p_min'][-1].size) 
        exp_tard = (indices * self.system['p_min'][-1]).sum()
        lowest_loss = omega * test_wt / self.parameters['x'].sum() + (1 - omega) * exp_tard
        store_optim = dict({ 'x' : self.parameters['x'].copy(), 'system' : copy.deepcopy(self.system), 'tot_wt' : test_wt})
        
        # Continue the search until no improvement is found
        while True:  # Start an outer loop that will continue until explicitly broken
            nh = generate_search_neighborhood(self.parameters['x'])  # Generate a new neighborhood
            improved = False  # Flag to check if an improvement was found in the inner loop
            
            for y in nh:  # Inner loop to search through the neighborhood
                # Insert first element of nh in x
                self.parameters['x'] = y.copy()
                logging.info(f"Test schedule = {self.parameters['x']}")
                # Set starting state
                self.state = 0
                # If start state is 0 reinitialize p_min, w and p_plus in state 0
                self._initialize_system()
                self.calculate_system_states(until=len(self.parameters['x']))
                logging.info("System recalculated")
                self.calculate_wait_times()
                test_wt = self.system["ew"].sum()
                logging.info(f"Average waiting time={test_wt / self.parameters['x'].sum()}")
                self.calculate_tardiness()
                indices = np.arange(self.system['p_min'][-1].size) 
                exp_tard = (indices * self.system['p_min'][-1]).sum()
                logging.info(f'Expected tardiness={exp_tard}')
                test_loss = omega * test_wt / self.parameters['x'].sum() + (1 - omega) * exp_tard
                logging.info(f'obj_value = {test_loss}')
                # If a neighborhood with a lower waiting time is found, store the system
                if(test_loss < lowest_loss):
                    lowest_loss = test_loss
                    store_optim['x'] = self.parameters['x'].copy()
                    newsystem = copy.deepcopy(self.system)
                    store_optim['system'] = newsystem
                    store_optim['tot_wt'] = test_wt
                    logging.info(f'{datetime.datetime.now()} - Found lower loss = {lowest_loss} with x = {store_optim["x"]} and system = {store_optim["system"]}')
                    improved = True  # Set the flag because an improvement was found
                    break  # Exit the inner loop to generate a new neighborhood
            
            if not improved:  # If no improvement was found in the inner loop
                logging.info(f'Finished searching')
                # Set the system to the stored optimal system 
                logging.info(f'{datetime.datetime.now()} - Final result lowest loss = {lowest_loss} with x = {store_optim["x"]} and system = {store_optim["system"]}')
                self.parameters['x'] = store_optim['x'].copy()
                self.system['p_min'] = store_optim['system']['p_min']
                self.system['p_plus'] = store_optim['system']['p_plus']
                self.system['w'] = store_optim['system']['w']
                self.system['ew'] = store_optim['system']['ew']
                break  # Exit the outer loop - the search is complete
      
        
    def small_local_search(self):
          """
          Generate for a given schedule a local search environment, establish for each schedule in the environment the lowest total waiting time, if a schedule with a lower waiting time is found, use this schedule to generate a new local environment and search this environment for lower waiting times. When a local environment contains no schedules with lower waiting stop searching.  
          """
          omega = self.parameters['omega']
          # Calculate initial loss
          test_wt = self.system["ew"].sum()
          self.calculate_tardiness()
          indices = np.arange(self.system['p_min'][-1].size) 
          exp_tard = (indices * self.system['p_min'][-1]).sum()
          lowest_loss = omega * test_wt / self.parameters['x'].sum() + (1 - omega) * exp_tard
          store_optim = dict({ 'x' : self.parameters['x'].copy(), 'system' : copy.deepcopy(self.system), 'tot_wt' : test_wt})
        
          # Continue the search until no improvement is found
          while True:  # Start an outer loop that will continue until explicitly broken
              nh = generate_small_search_neighborhood(self.parameters['x'])  # Generate a new neighborhood
              improved = False  # Flag to check if an improvement was found in the inner loop
              
              for y in nh:  # Inner loop to search through the neighborhood
                  # Insert first element of nh in x
                  self.parameters['x'] = y.copy()
                  logging.info(f"Test schedule = {self.parameters['x']}")
                  # Set starting state
                  self.state = 0
                  # If start state is 0 reinitialize p_min, w and p_plus in state 0
                  if(self.state == 0):
                      self.system = dict({
                          'p_min': np.zeros((len(self.parameters['x']) + 1, len(self.parameters['s'])), dtype=np.float64),
                          'p_plus': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64)
                      })
                      # Set the first element of p_min in the initial state to 1.
                      self.system['p_min'][0][0] = 1
                      # Initialize array of arrays for saving waiting times distributions per patient
                      self.system['w'] = zero_arrays(self.parameters['x'].copy(), len(self.parameters['s']))
                      if(self.parameters['x'][0] > 0): # Only calculate waiting times if there are patients scheduled in the state
                               self.system['w'][0][0] = self.system['p_min'][0].copy()
                               for i in range(1, self.parameters['x'][0]):
                                      self.system['w'][0][i] = np.convolve(self.system['w'][0][i-1], self.parameters['s'])[:(len(self.parameters['s']))]
                      self.system['p_plus'][0] = self.system['p_min'][0].copy() if self.parameters['x'][0] == 0 else np.convolve(self.system['w'][0][-1], self.parameters['s'])[:(len(self.parameters['s']))]
                      # Initialize array for saving total expected waiting times per state
                      self.system['ew'] = np.zeros(len(self.parameters['x']), dtype=np.float64)
                      # Set the initial state to 1.
                      self.state = 1
                  self.calculate_system_states(until=len(self.parameters['x']))
                  logging.info("System recalculated")
                  self.calculate_wait_times()
                  test_wt = self.system["ew"].sum()
                  logging.info(f"Average waiting time={test_wt / self.parameters['x'].sum()}")
                  self.calculate_tardiness()
                  indices = np.arange(self.system['p_min'][-1].size) 
                  exp_tard = (indices * self.system['p_min'][-1]).sum()
                  logging.info(f'Expected tardiness={exp_tard}')
                  test_loss = omega * test_wt / self.parameters['x'].sum() + (1 - omega) * exp_tard
                  logging.info(f'obj_value = {test_loss}')
                  # If a schedule with a lower waiting time is found, store the system
                  if(test_loss < lowest_loss):
                    lowest_loss = test_loss
                    store_optim['x'] = self.parameters['x'].copy()
                    newsystem = copy.deepcopy(self.system)
                    store_optim['system'] = newsystem
                    store_optim['tot_wt'] = test_wt
                    logging.info(f'{datetime.datetime.now()} - Found lower loss = {lowest_loss} with x = {store_optim["x"]}')
                    improved = True  # Set the flag because an improvement was found
                    break  # Exit the inner loop to generate a new neighborhood
              
              if not improved:  # If no improvement was found in the inner loop
                logging.info(f'Finished searching')
                # Set the system to the stored optimal system 
                logging.info(f'{datetime.datetime.now()} - Final result lowest loss = {lowest_loss} with x = {store_optim["x"]} and system = {store_optim["system"]}')
                self.parameters['x'] = store_optim['x'].copy()
                self.system['p_min'] = store_optim['system']['p_min']
                self.system['p_plus'] = store_optim['system']['p_plus']
                self.system['w'] = store_optim['system']['w']
                self.system['ew'] = store_optim['system']['ew']
                break  # Exit the outer loop - the search is complete
          
    def calculate_tardiness(self):
      """
      Calculate the tardiness for the current system and add it as the last element to p_min - the distribution of work in interval t just before any patient has arrived. 
      """
      self.system['p_min'][-1][0] = np.sum(self.system['p_plus'][-1][:(self.parameters['d'] + 1)])
      self.system['p_min'][-1][1:(-1*self.parameters['d'])] = self.system['p_plus'][-1][(self.parameters['d'] + 1):]
      # logging.info(f'{datetime.datetime.now()} - calculating tardiness distribution:  {self.system["p_min"][-1]}')
      
    
    def calculate_wait_times(self):
      """
      Calculate the expected waiting time for each time step.
      """
      for interval, wtdists in enumerate(self.system['w']):
          ew = 0
          for nr, dist in enumerate(wtdists):
              # Calculate the weighted sum of the waiting time distribution
              a = range(len(dist))
              b = dist
              meanwt = np.dot(a, b)
              logging.info(f"Mean waiting time for patient {nr} in interval {interval} = {meanwt}")
              ew += meanwt
          # Store the expected waiting time for the current time step
          self.system['ew'][interval] = ew
        
        
    def visualize_state(self, state='', dist='p_plus'):
        """
        Visualize a distribution in the last given state.
        
        Args:
        state (int): An optional integer representing the state to visualize. If not provided, the current state is used.
        dist (str): An optional string representing the distribution to visualize. Defaults to 'p_plus'.
        """
        if state == '':
            state = self.state
        trace = go.Scatter(
            x=list(range(len(self.system[dist][state]))),
            y=self.system[dist][state],
            mode='lines'
        )
        
        data = [trace]
        
        layout = go.Layout(
            title=f'{dist} in state {state} with x = {self.parameters["x"]}<br><sub>AUR = {np.round(np.sum(self.system[dist][state]), 3)}</sub>',
            xaxis=dict(title='Time'),
            yaxis=dict(title='Probability')
        )
        
        fig = go.Figure(data=data, layout=layout)
        pyo.iplot(fig)
    
    def visualize_system(self, dist='p_plus'):
        l = len(self.parameters["x"])
        suffix = ""
        if(dist=='p_min'): suffix = f'<br>NB: p_min[{l}] contains tardiness distribution'
        """
        Visualize a distribution for the entire system.
        
        Args:
        dist (str): An optional string representing the distribution to visualize. Defaults to 'p_plus'.
        """
        values = self.system[dist]
        trace = go.Heatmap(
                    z=values,
                    colorscale=[
                      [0, 'white'],  # white color at the bottom
                      [1, 'red']     # red color at the top
                    ]
                    )
        data = [trace]
        layout = go.Layout(
            title=f'{dist} with x = {self.parameters["x"]}{suffix}',
            xaxis=dict(title='Time'),
            yaxis=dict(title='Interval')
        )
        fig = go.Figure(data=data, layout=layout)
        pyo.iplot(fig)
        
    def __str__(self):
        """
        Print all probabilities
        """
        return("p_min = % s \nw = % s \np_plus = % s \new = % s" % (self.system['p_min'], self.system['w'], self.system['p_plus'], self.system['ew']))

This Python class, named **`Schedule`**, simulates a scheduling system. Itis be used for analyzing a scheduling system that has time intervals with varying numbers of scheduled patients.

Here's a breakdown of what happens in the class:

1.  **Initialization (`__init__`):** In the initialization function, parameters **`x`**, **`d`**, **`s`** are set which represent the patients scheduled in each interval, the interval length, and the service time distribution, respectively. It also initializes certain elements of the **`system`** dictionary such as **`p_min`**, **`p_plus`**, and **`w`** which are used for storing the distributions of work in the system at the start and end of each state and the distributions of waiting times of each patient.

2.  **`calculate_system_states`:** This function is used to calculate the system states. This involves calculation of the amount of work left in the system just before the start of each state (**`p_min`**) and just before the end of each state (**`p_plus`**), along with the waiting time distributions of patients (**`w`**) using convolution. The **`until`** parameter can be used to control until which state the system states should be calculated.

3.  **`impute_compositions`:** This function changes the scheduling of patients (represented by the array **`x`**) for a certain number of intervals (**`k`** intervals starting from interval **`s`**). All possible ways of distributing a certain number of patients (**`n`**) over the intervals are generated (**`compositions`**), for each possible distribution the system states are recalculated and the schedule with the lowest total waiting time is established. *NB: The system only requires recalculation starting from interval 's', because the schedule has not changed before that that interval.*

4.  **`search_function(k)`:** This function invokes `impute_compositions()` for different start states to find the optimal schedule with the lowest waiting time.

5.  **`calculate_wait_times`:** This function calculates the expected waiting times for each interval by adding up the expected waiting times of all tasks in the interval.

6.  **`visualize_state` and `visualize_system`:** These functions are used to visualize a certain distribution (**`p_min`**, **`p_plus`**, or **`w`**) for a certain state or for all states in a heat map. The visualization is done using Plotly.

7.  **`__str__`:** This function is used to get a string representation of the schedule instance, which shows the values of the **`p_min`**, **`w`**, **`p_plus`**, and **`ew`** arrays.


In [ ]:
def run_schedule(x, d, s, q, omega, print_system=True):
    schedule = Schedule(x=x, d=d, s=s, q=q, omega=omega)
    schedule.calculate_system_states(until=len(x))
    schedule.calculate_wait_times()
    schedule.calculate_loss()
    if(print_system): print(schedule)
    return(schedule)

In [ ]:
"""Simple test case"""
d = 3
q = 0.0
omega = 0.6
s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float64)
# s= service_time_with_no_shows(s, q)
indices = np.arange(s.size)
exp_s = (indices * s).sum()
print(f'exp_s = {exp_s}')

In [ ]:
N = 6
schedules = generate_schedules(N)
data = {'x0': [], 'x1': [], 'x2': [], 'loss': []}
for schedule in schedules:
  x = np.array(schedule, dtype=np.int64)
  sch = run_schedule(x, d, s, q, omega, False)
  x0, x1, x2 = x
  data['x0'].append(x0)
  data['x1'].append(x1)
  data['x2'].append(x2)
  data['loss'].append(sch.system['loss'])

df = pd.DataFrame.from_dict(data)
df

In [ ]:
# Create a 3D scatter plot with Plotly with arrows

# Sort the dataframe by 'loss' to find the 5th lowest and the lowest loss
sorted_df = df.sort_values(by='loss').reset_index(drop=True)

# Create the 3D scatter plot using Plotly Graph Objects
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter3d(
    x=df['x0'], y=df['x1'], z=df['x2'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['loss'],
        colorscale='matter',
        showscale=False  # Hide the colorbar)
    ),
    text=df['loss'],
    hovertemplate='[%{x}, %{y}, %{z}] - loss = %{text:.2f}',
))

# Add red arrows from the 5th lowest loss to the lowest loss point
for i in range(4):
    fig.add_trace(go.Scatter3d(
        x=[sorted_df['x0'][i], sorted_df['x0'][i + 1]],
        y=[sorted_df['x1'][i], sorted_df['x1'][i + 1]],
        z=[sorted_df['x2'][i], sorted_df['x2'][i + 1]],
        mode='lines',
        line=dict(color='red', width=4),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title='Solution landscape for schedule with N = {} and T = 3'.format(N),
    scene=dict(
        xaxis_title='x0',
        yaxis_title='x1',
        zaxis_title='x2',
    ),
    width=800,
    height=800,
    showlegend=False
)
#fig.write_image("images/landscape-with-arrows.png", scale=4)
fig.show()

In [ ]:
# Create a 3D scatter plot with Plotly with arrows

# Sort the dataframe by 'loss' to find the 5th lowest and the lowest loss
sorted_df = df.sort_values(by='loss').reset_index(drop=True)

# Create the 3D scatter plot using Plotly Graph Objects
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter3d(
    x=df['x0'], y=df['x1'], z=df['loss'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['loss'],
        colorscale='matter',
        showscale=False  # Hide the colorbar)
    ),
    text=df['loss'],
    hovertemplate='[%{x}, %{y}, %{z}] - loss = %{text:.2f}',
))

# Add red arrows from the 5th lowest loss to the lowest loss point
for i in range(4):
    fig.add_trace(go.Scatter3d(
        x=[sorted_df['x0'][i], sorted_df['x0'][i + 1]],
        y=[sorted_df['x1'][i], sorted_df['x1'][i + 1]],
        z=[sorted_df['loss'][i], sorted_df['loss'][i + 1]],
        mode='lines',
        line=dict(color='red', width=4),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title='Solution landscape for schedule with N = {} and T = 3'.format(N),
    scene=dict(
        xaxis_title='x0',
        yaxis_title='x1',
        zaxis_title='loss',
    ),
    width=800,
    height=800,
    showlegend=False
)
fig.write_image("images/landscape-with-arrows.png", scale=4)
fig.show()